In [1]:
import pandas as pd
import numpy as np

import yfinance as yf

/Users/markhendricks/Projects/finm-portfolio-2024/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
TICKS = [
    'SPY',
    'EFA',
    'EEM',
    'PSP',
    'QAI',
    'HYG',
    'DBC',
    'IYR',
    'IEF',
    'BWX',
    'TIP',
    'SHV',
]

FLDS = ['shortName','quoteType','currency','volume','totalAssets','longBusinessSummary']

In [3]:
info = pd.DataFrame(index=TICKS,columns=FLDS)
info.index.name = 'ticker'
for tick in info.index:
    temp = yf.Ticker(tick).get_info()

    for fld in FLDS:
        if fld in temp.keys():
            info.loc[tick,fld] = temp[fld]

In [4]:
info

,shortName,quoteType,currency,volume,totalAssets,longBusinessSummary
ticker,,,,,,
SPY,SPDR S&P 500,ETF,USD,34043802,566341992448,The Trust seeks to achieve its investment obje...
EFA,iShares MSCI EAFE ETF,ETF,USD,8331631,57923706880,The fund generally will invest at least 80% of...
EEM,iShares MSCI Emerging Index Fun,ETF,USD,55091381,17685696512,The fund generally will invest at least 80% of...
PSP,Invesco Global Listed Private E,ETF,USD,7690,248054672,The fund generally will invest at least 90% of...
QAI,NYLI Hedge Multi-Strategy Track,ETF,USD,86881,574961600,"The fund is a ""fund of funds"" which means it i..."
HYG,iShares iBoxx $ High Yield Corp,ETF,USD,41017932,16903942144,The underlying index is a rules-based index co...
DBC,Invesco DB Commodity Index Trac,ETF,USD,1089789,1496658176,The fund pursues its investment objective by i...
IYR,iShares U.S. Real Estate ETF,ETF,USD,5505016,4560725504,The fund seeks to track the investment results...
IEF,iShares 7-10 Year Treasury Bond,ETF,USD,7739016,31752601600,The underlying index measures the performance ...


In [5]:
STARTDATE = '2011-01-01'
ENDDATE = '2024-09-30'

tickers = list(info.index.values)
df = yf.download(tickers, start=STARTDATE, end=ENDDATE)['Adj Close']

[*********************100%***********************]  12 of 12 completed


In [6]:
prices = df.resample('M').last()

# eliminate timezones
prices.index = prices.index.tz_localize(None)

rets = prices.pct_change().dropna()

# change to excess returns, in excess of short-term treasury
retsx = rets.subtract(rets['SHV'], axis=0)
retsx = retsx.drop(columns=['SHV'])

/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_32743/3831157974.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  prices = df.resample('M').last()


In [7]:
rets

Ticker,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SHV,SPY,TIP
Date,,,,,,,,,,,,
2009-04-30,0.008993,-0.001000,0.155582,0.115190,0.138460,-0.027452,0.296151,0.230203,0.022882,0.000553,0.099346,-0.017952
2009-05-31,0.053672,0.162663,0.159400,0.131918,0.028555,-0.020773,0.022727,0.053892,0.027865,-0.000472,0.058453,0.019967
2009-06-30,0.005149,-0.026259,-0.022495,-0.014049,0.033518,-0.005572,-0.024863,0.045449,-0.003436,0.000599,-0.000655,0.001980
2009-07-31,0.031284,0.018568,0.110146,0.100414,0.069190,0.008317,0.105799,0.143247,0.015325,-0.000027,0.074606,0.000880
2009-08-31,0.007628,-0.040365,-0.013136,0.045031,-0.016969,0.007635,0.131939,0.033413,-0.004151,0.000435,0.036940,0.008413
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31,0.014228,-0.002999,0.019517,0.050602,0.016339,0.018001,0.049274,0.039039,0.013434,0.004581,0.050580,0.017770
2024-06-30,-0.009475,-0.001719,0.026180,-0.018267,0.004794,0.012170,0.018764,-0.017107,0.003233,0.004199,0.035280,0.007615
2024-07-31,0.038512,-0.027981,0.008453,0.025916,0.023522,0.028972,0.076248,0.068587,0.011279,0.004599,0.012109,0.017274


In [8]:
retsx

Ticker,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Date,,,,,,,,,,,
2009-04-30,0.008441,-0.001553,0.155029,0.114637,0.137907,-0.028005,0.295598,0.229650,0.022330,0.098794,-0.018505
2009-05-31,0.054143,0.163134,0.159872,0.132390,0.029026,-0.020301,0.023198,0.054364,0.028337,0.058925,0.020439
2009-06-30,0.004550,-0.026858,-0.023094,-0.014648,0.032919,-0.006171,-0.025462,0.044850,-0.004035,-0.001254,0.001381
2009-07-31,0.031311,0.018594,0.110173,0.100441,0.069217,0.008344,0.105826,0.143274,0.015352,0.074632,0.000907
2009-08-31,0.007193,-0.040800,-0.013571,0.044595,-0.017404,0.007199,0.131504,0.032977,-0.004586,0.036505,0.007978
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31,0.009647,-0.007581,0.014935,0.046020,0.011758,0.013420,0.044692,0.034457,0.008852,0.045998,0.013189
2024-06-30,-0.013674,-0.005918,0.021982,-0.022466,0.000595,0.007971,0.014565,-0.021305,-0.000965,0.031082,0.003417
2024-07-31,0.033913,-0.032580,0.003854,0.021317,0.018924,0.024374,0.071649,0.063989,0.006681,0.007511,0.012676


In [9]:
with pd.ExcelWriter('../data/multi_asset_etf_data.xlsx') as writer:  
    info.to_excel(writer, sheet_name= 'descriptions')
    prices.to_excel(writer, sheet_name= 'prices')
    rets.to_excel(writer, sheet_name='total returns')
    retsx.to_excel(writer, sheet_name='excess returns')